In [ ]:
from csv import DictReader
from random import Random
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
from keras.utils.np_utils import to_categorical
import pickle as pkl
import warnings
warnings.filterwarnings('ignore')

In [ ]:
RANDOM = Random(42)

## Load Data

In [ ]:
def load_samples(path):
    with open(path) as f:
        samples = list(DictReader(f))
        for sample in samples:
            sample['target'] = int(sample.get('target', -1))
        return samples

In [ ]:
def train_val_split(samples, split=0.2):
    RANDOM.shuffle(samples)
    n_val = int(len(samples) * split)
    return samples[:-n_val], samples[-n_val:]

In [ ]:
import pandas as pd
dat_prot = pd.read_table('/Users/suhancho/data/Uniprot_metalbinding_challenge/sequence_df.tsv')
chebi = pd.read_table('/Users/suhancho/data/Uniprot_metalbinding_challenge/POS_TRAIN_FULL.tsv')

In [ ]:
dat_prot_bindsite = pd.merge(dat_prot,chebi,left_on = 'Protein name',right_on='Accession',how = 'outer')
neg_seqs = dat_prot_bindsite[dat_prot_bindsite.Type=='neg_sequence']
neg_seqs['Position'] = [int(divmod(len(p),2)[0]) for p in neg_seqs['Protein sequence'].tolist()]
neg_seqs['ChEBI-ID'] = 'NB'
dat_prot_bindsite = pd.concat([dat_prot_bindsite[dat_prot_bindsite['Type']=='pos_sequence'],neg_seqs]).reset_index(drop = True)
dat_prot_bindsite['ChEBI-ID'] = pd.Categorical(dat_prot_bindsite['ChEBI-ID'])
dat_prot_bindsite['target'] = dat_prot_bindsite['ChEBI-ID'].cat.codes
dat_prot_bindsite['Position'] = dat_prot_bindsite['Position'].astype(int)
dat_prot_bindsite_sampled = dat_prot_bindsite.sample(frac=1,random_state=9510)
# dat_prot_bindsite_sampled.to_csv('/Users/suhancho/data/Uniprot_metalbinding_challenge/data_before_windowing.tsv',sep='\t')

In [ ]:
def get_windowdf(protfile,winsize=4):
    window=[]
    with open(protfile,'r') as p:
        for line in tqdm(p):
            if not line.count('Protein sequence'):
                protseq = line.split('\t')[2].strip()
                bindsite = int(line.split('\t')[-2].strip())

                if ((bindsite-winsize)<0 )& ((bindsite+winsize) > len(protseq)) :
                    front_pad_seq = 'X'*(winsize-bindsite)
                    bindseq = protseq[bindsite-winsize : bindsite+winsize]
                    rear_pad_seq = 'X'*((winsize+bindsite)-len(protseq))
                    windowed = front_pad_seq+bindseq+rear_pad_seq

                elif ((bindsite-winsize)<0) & ((bindsite+winsize) < len(protseq)):
                    front_pad_seq = 'X'*(winsize-bindsite)
                    bindseq = protseq[bindsite-winsize : bindsite+winsize]
                    windowed = front_pad_seq+bindseq

                elif ((bindsite-winsize)>0) & ((bindsite+winsize) > len(protseq)):
                    rear_pad_seq = 'X'*((winsize+bindsite)-len(protseq))
                    bindseq = protseq[bindsite-winsize : bindsite+winsize]
                    windowed = bindseq+rear_pad_seq

                else:
                    windowed = protseq[bindsite-winsize : bindsite+winsize]
                    
                window.append(windowed)
    return(window)

In [ ]:
test = get_windowdf('/Users/suhancho/data/Uniprot_metalbinding_challenge/data_before_windowing.tsv',winsize = 4)
dat_prot_bindsite_sampled['window_4'] = test
dat_prot_bindsite_sampled = dat_prot_bindsite_sampled.reset_index()
dat_prot_bindsite_sampled.rename(columns = {'index':'qid','window_4':'question_text'},inplace = True)
dat_prot_bindsite_sampled['qid'] = 'HASH_'+dat_prot_bindsite_sampled['qid'].astype(str)
one_hot_labels = to_categorical(dat_prot_bindsite_sampled['target'].tolist())
dat_prot_bindsite_sampled['target'] = list(one_hot_labels)
samples=dat_prot_bindsite_sampled[['qid','question_text','target']].to_dict('records')
train_samples, val_samples = train_val_split(samples)

## Preprocessing

In [ ]:
def build_vocabulary(samples, vocab_min_freq=100):
    counts = Counter(ch for sample in samples for ch in sample['question_text'])
    chars = sorted(ch for ch, count in counts.items() if count >= vocab_min_freq)
    return {char: i for i, char in enumerate(chars)}

In [ ]:
vocabulary = build_vocabulary(train_samples)
# print(len(vocabulary))
# vocabulary

In [ ]:
def transform(sample, vocabulary):
    sample['encoded_text'] = [vocabulary[ch] for ch in sample['question_text'] if ch in vocabulary]
    return sample

In [ ]:
train_samples = [transform(sample, vocabulary) for sample in train_samples]
val_samples = [transform(sample, vocabulary) for sample in val_samples]

## Modeling

In [ ]:
import keras
from keras import backend as K
from keras import initializers, regularizers, constraints
# from keras.engine import Layer
# from tensorflow.keras.layers import Layer
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
# from keras.layers import InputLayer, Input

class AttentionWithContext(keras.layers.Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.

    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.

    Note: The layer has been tested with Keras 2.0.6

    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape = (input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape = (input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape = (input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [ ]:
from keras import models, Model
from keras.layers import Input, Embedding, Conv1D, Add, Dense, SpatialDropout1D

def _conv_block(x, filters, kernel_size):
    conv = Conv1D(filters, kernel_size, activation='relu', padding='same')(x)
    conv = Conv1D(filters, kernel_size, activation='relu', padding='same')(conv)
    return conv


def _resblock(x, filters, kernel_size):
    conv = _conv_block(x, filters, kernel_size)
    projection = Conv1D(filters, 1, padding='same')(x)
    return Add()([conv, projection])


def predict_bindsite(vocab_size,
                    char_embedding_size,
                    base_filters,
                    doc_embedding_size,
                    dropout):
    text = Input(shape=(None,))
    embedding = Embedding(vocab_size, char_embedding_size)(text)

    conv_1 = _resblock(embedding, base_filters, 3)
    conv_1 = SpatialDropout1D(dropout)(conv_1)
    conv_2 = _resblock(conv_1, base_filters * 2, 3)
    conv_2 = SpatialDropout1D(dropout)(conv_2)
    conv_3 = _resblock(conv_2, base_filters * 4, 3)
    conv_3 = SpatialDropout1D(dropout)(conv_3)
    conv_4 = _resblock(conv_3, base_filters * 8, 3)
    conv_4 = SpatialDropout1D(dropout)(conv_4)

    attention = AttentionWithContext()(conv_4)

    fc_1 = Dense(doc_embedding_size, activation='relu')(attention)
    fc_2 = Dense(doc_embedding_size, activation='relu')(fc_1)
    # prediction = Dense(1, activation='sigmoid')(fc_2)
    # prediction = Dense(29, activation='sigmoid')(fc_2)
    prediction = Dense(30, activation='sigmoid')(fc_2) # Originally sigmoid

    model = Model(text, prediction)
    # model.compile('adam', 'binary_crossentropy', metrics=['acc'])
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


    return model

## Training Support

In [ ]:
from math import ceil

class BatchProvider:
    def __init__(self, samples, batch_size, shuffle=False, run_forever=False):
        self._samples = samples
        self._batch_size = batch_size
        self._shuffle = shuffle
        self._run_forever = run_forever

    def generate_batches(self):
        batch = []
        indices = list(range(len(self._samples)))
        while True:
            if self._shuffle:
                RANDOM.shuffle(indices)
            for i in indices:
                batch.append(self.get_item(i))
                if len(batch) == self._batch_size:
                    yield self.transform_batch(batch)
                    batch = []
            if not self._run_forever:
                break
        if batch:
            yield self.transform_batch(batch)

    def __len__(self):
        return int(ceil(len(self._samples) / self._batch_size))

    def get_item(self, idx):
        sample = self._samples[idx]
        return sample['encoded_text'], sample['target']

    def transform_batch(self, items):
        texts, targets = zip(*items)
        max_length = max(len(text) for text in texts)
        text_batch = np.zeros((len(texts), max_length))
        for i, text in enumerate(texts):
            text_batch[i, :len(text)] = text
        target_batch = np.array(targets)
        return text_batch, target_batch

## Inference Support

In [ ]:
from math import ceil

class TestBatchProvider:
    def __init__(self, samples, batch_size, shuffle=False, run_forever=False):
        self._samples = samples
        self._batch_size = batch_size
        self._shuffle = shuffle
        self._run_forever = run_forever

    def generate_batches(self):
        batch = []
        indices = list(range(len(self._samples)))
        while True:
            if self._shuffle:
                RANDOM.shuffle(indices)
            for i in indices:
                batch.append(self.get_item(i))
                if len(batch) == self._batch_size:
                    yield self.transform_batch(batch)
                    batch = []
            if not self._run_forever:
                break
        if batch:
            yield self.transform_batch(batch)

    def __len__(self):
        return int(ceil(len(self._samples) / self._batch_size))

    def get_item(self, idx):
        sample = self._samples[idx]
        return sample['encoded_text']

    def transform_batch(self, items):
        texts= items
        max_length = max(len(text) for text in texts)
        text_batch = np.zeros((len(texts), max_length))
        for i, text in enumerate(texts):
            text_batch[i, :len(text)] = text
        return text_batch

## Training

In [ ]:
model = predict_bindsite(
    vocab_size=len(vocabulary),
    char_embedding_size=16,
    base_filters=32,#32
    doc_embedding_size=300,
    dropout=0.1
)
model.summary()

In [ ]:
train_batch_provider = BatchProvider(train_samples, batch_size=2048, shuffle=True, run_forever=True)
train_batches = train_batch_provider.generate_batches()
val_batch_provider = BatchProvider(val_samples, batch_size=2048, shuffle=False, run_forever=True)
val_batches = val_batch_provider.generate_batches()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
EPOCHS = 10
model_checkpoint_callback = ModelCheckpoint('./weights.hdf5',monitor='val_acc',save_best_only=True)

model.fit_generator(
        generator=train_batches,
        steps_per_epoch=len(train_batch_provider),
        epochs=EPOCHS,
        validation_data=val_batches,
        validation_steps=len(val_batch_provider),
        callbacks= model_checkpoint_callback
    )

In [ ]:
# model.load_weights('./weights.hdf5')
# model.save('./Saved_Ionbind_NLP_221210')
model = keras.models.load_model("Saved_Ionbind_NLP_221210")

## Calibration

In [ ]:
from sklearn.metrics import classification_report

val_batch_provider = BatchProvider(val_samples, batch_size=64, shuffle=False, run_forever=False)
val_predictions = model.predict_generator(val_batch_provider.generate_batches(), steps=len(val_batch_provider))
val_targets = np.array([sample['target'] for sample in val_samples])
val_prediction_argmax = [np.argmax(p) for p in val_predictions]
val_targets_argmax = [np.argmax(p) for p in val_targets]

report = classification_report(val_targets_argmax,val_prediction_argmax,output_dict = True)
report = pd.DataFrame(report).T

# Target ion information processing and result processing

In [ ]:
target_ion_info = dat_prot_bindsite_sampled[['target','ChEBI-ID']]
target_ion_info['target'] = target_ion_info['target'].apply(np.argmax).astype(int)
target_ion_info.drop_duplicates(inplace = True)

In [ ]:
report = classification_report(val_targets_argmax,val_prediction_argmax,output_dict = True)
report = pd.DataFrame(report)
report_class = report.drop(['accuracy','macro avg','weighted avg'],axis=1).T
report_class = report_class.reset_index()
report_class['index'] = report_class['index'].astype(int)
report_class = pd.merge(report_class,target_ion_info,left_on = 'index',right_on = 'target').drop('index',axis=1)
report_class = report_class[['target','ChEBI-ID','precision','recall','f1-score','support']]

In [ ]:
report_class.head(2)

## Submission

In [ ]:
from keras.utils import GeneratorEnqueuer
import csv

In [ ]:
# test_dat = pd.read_table('/Users/suhancho/data/Uniprot_metalbinding_challenge/test_sequence_df.tsv')
# test_dat['Protein name'] = 'HASH_'+test_dat['Protein name'].astype(str)
# test_dat.rename(columns = {'Protein name':'qid'},inplace = True)

# Inference

In [84]:
inference_result=[]
with open('/Users/suhancho/data/Uniprot_metalbinding_challenge/test_sequence_df.tsv','r') as testfile:
    for j,t in tqdm(enumerate(testfile)):
        if not t.count('Protein'):
            test_input_lst=[]
            testseq_tmp = t.split('\t')[1].strip()
            for i in list(range(len(testseq_tmp))):
                test_input={} # Generate Dictionary for prediciton
                testprot_tmp = 'HASH_'+str(t.split('\t')[2].strip())+'.'+str(i)
                if i-4<0:
                    windowed_tmp = 'X'*(4-i)+testseq_tmp[0:i+5]
                elif i+5>len(testprot_tmp):
                    windowed_tmp = testseq_tmp[i-4:i+5]+'X'*(i+5-len(testseq_tmp)+1)
                else : 
                    windowed_tmp = testseq_tmp[i-4:i+5]

                test_input['qid'] = testprot_tmp
                test_input['question_text'] = windowed_tmp
                test_input_lst.append(test_input)

            test_input_transformed = [transform(sample, vocabulary) for sample in test_input_lst]

            sample_ids = (sample['qid'] for sample in test_input_lst)
            test_batch_provider = TestBatchProvider(test_input_lst, batch_size=len(test_input_lst), shuffle=False, run_forever=False)
            enqueuer = GeneratorEnqueuer(test_batch_provider.generate_batches())
            enqueuer.start()
            test_batches = enqueuer.get()
            for batch in test_batches:
                test_predictions = model.predict_on_batch(batch)
                test_prediction_argmax = [np.argmax(p) for p in test_predictions]
                test_prediction_proba = [prob[idx] for prob,idx in zip(test_predictions,test_prediction_argmax)]
                inference_result.append([testprot_tmp.split('.')[0].lstrip('HASH_'),test_prediction_argmax,test_prediction_proba])

32it [00:03,  8.52it/s]


KeyboardInterrupt: 

In [85]:
pd.DataFrame(inference_result)

,0,1,2
0,0573999,"[2, 29, 29, 29, 29, 29, 29, 29, 14, 29, 29, 29...","[0.6886938, 0.9692102, 0.98517925, 0.99607575,..."
1,0569043,"[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 2...","[0.9330529, 0.9998642, 0.9996815, 0.99917895, ..."
2,0551880,"[26, 29, 29, 29, 29, 2, 11, 29, 29, 2, 14, 29,...","[0.8498795, 0.99403894, 0.91398317, 0.9372454,..."
3,0257927,"[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 2...","[0.52239823, 0.9998512, 0.992098, 0.97448486, ..."
4,0579555,"[29, 29, 29, 29, 2, 29, 29, 29, 29, 29, 29, 29...","[0.88516253, 0.992071, 0.98965925, 0.99695176,..."
5,1011827,"[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 1...","[0.81877303, 0.99829227, 0.99721175, 0.997317,..."
6,0240162,"[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 2...","[0.8039206, 0.9973279, 0.9967237, 0.95503867, ..."
7,1003604,"[2, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29...","[0.97470576, 0.99711055, 0.99036497, 0.9862323..."
8,1006062,"[29, 29, 29, 29, 29, 2, 29, 29, 29, 29, 29, 29...","[0.8491071, 0.9980323, 0.98971343, 0.955023, 0..."
9,0864283,"[2, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29...","[0.9309748, 0.9939902, 0.98760265, 0.96765125,..."
